In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import pandas as pd
import numpy as np
import torch
import itertools
import numpy as np
import scipy.sparse as sp
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import EGATConv
from startup_data_set import *
from PredictorClasses import *
from CustomMetrics import *
from CustomUtilities import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [2]:
graph = COMP4222Dataset()
g = graph[0].to(device)

In [3]:
u,v=g.edges()
print(u)
print(v)

tensor([  203,   231,   232,  ...,  5030,  8850, 15490])
tensor([17852, 17852, 17852,  ..., 25444, 25444, 25445])


In [4]:
g.ndata["feat"].shape[1]

221

In [5]:
train_g, train_pos_g, val_pos_g, test_pos_g = generate_pos_graph(g)

In [6]:
train_neg_g, val_neg_g, test_neg_g = generate_neg_graph(g)

In [7]:
train_g, train_pos_g, val_pos_g, test_pos_g = train_g.to(device), train_pos_g.to(device), val_pos_g, test_pos_g.to(device)
train_neg_g, val_neg_g, test_neg_g = train_neg_g.to(device), val_neg_g.to(device), test_neg_g.to(device)
num_nodes = g.num_nodes
num_edges = g.num_edges

In [8]:
egat_model = EGATConv(in_node_feats = train_g.ndata["feat"].shape[1],
                in_edge_feats = train_g.edata["feat"].shape[1],
                out_node_feats = train_g.ndata["feat"].shape[1],
                out_edge_feats = train_g.edata["feat"].shape[1],
                num_heads = 3).to(device)

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats , 64).to(torch.float32)
        self.W2 = nn.Linear(64, 1).to(torch.float32)

    # concat the source and destination node, use mlp to predict the score
    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1).to(torch.float32)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h, e):
        with g.local_scope():
            g.ndata['h'] = h.to(torch.float32)
            g.edata['feat'] = e.to(torch.float32)
            g.apply_edges(self.apply_edges)
            return g.edata['score']


pred = MLPPredictor(train_g.ndata["feat"].shape[1])



In [13]:
import itertools

train_loss, val_loss = [], []
train_AUC, val_AUC = [], []
stop = 0
ep = 100
optimizer = torch.optim.Adam(itertools.chain(egat_model.parameters(), pred.parameters()))
for e in range(ep):
    # forward
    h, e= egat_model(train_g, train_g.ndata['feat'].to(torch.float32), train_g.edata["feat"].to(torch.float32))
    pos_score = pred(train_pos_g,h,e)
    neg_score = pred(train_neg_g,h,e)
    print(pos_score.shape)
    print(neg_score.shape)
    loss = compute_loss(pos_score, neg_score)
    train_loss.append(loss.item())
    train_AUC.append(compute_auc(pos_score, neg_score))

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # validation
    v_pos_score = pred(val_pos_g, h)
    v_neg_score = pred(val_neg_g, h)
    v_loss = compute_loss(v_pos_score, v_neg_score)
    val_loss.append(v_loss.item())
    val_AUC.append(compute_auc(v_pos_score, v_neg_score))

    #verbose
    if e % 10 == 0:
        print('Epoch: {} \t Train loss: {} \t Val loss: {} \t Train AUC: {} \t Val AUC: {}'.format(e, round(loss.item(), 3), round(v_loss.item(), 3), round(train_AUC[-1],3), round(val_AUC[-1], 3)))


    # early stopping
    if e > 10:
        if v_loss.item() > sum(val_loss[-5:])/5:
            stop += 1
        else: 
            stop = 0
        if stop >= early_stopping:
            print("Early Stopped at Epoch {}".format(e))
            break

DGLError: Expect number of features to match number of edges. Got 61943 and 36497 instead.